In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

%matplotlib inline
pd.set_option('display.max_rows', 500)

![CRISP_DM](../reports/figures/CRISP_DM.png)

# Groupby apply on large (relational) data set

## Attentions all writen functions assume a data frame where the date is sorted!!

In [11]:
pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
pd_JH_data=pd_JH_data.sort_values('Date',ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()

,Unnamed: 0,Date,State,Country,Confirmed
0,0,2020-01-22,NaN,Afghanistan,0
1,308,2020-01-22,Zhejiang,Cuba,0
2,307,2020-01-22,Zhejiang,Croatia,0
3,306,2020-01-22,Zhejiang,Cote d'Ivoire,0
4,305,2020-01-22,Zhejiang,Costa Rica,0


# Test data

In [15]:
test_data=pd_JH_data[((pd_JH_data['Country']=='US')|
                      (pd_JH_data['Country']=='Germany'))&
                     (pd_JH_data['Date']>'2020-03-20')]


In [16]:
test_data.head()

,Unnamed: 0,Date,State,Country,Confirmed
26704,26998,2020-03-21,Zhejiang,Germany,22213
26823,27106,2020-03-21,Zhejiang,US,25600
26913,26732,2020-03-21,NaN,Germany,22213
27031,26840,2020-03-21,NaN,US,25600
27156,27450,2020-03-22,Zhejiang,Germany,24873


In [48]:
#test_data.groupby(['Country']).agg(np.max)

In [22]:
# %load ../src/features/build_features.py

import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)

def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope




In [24]:
test_data.groupby(['State','Country']).agg(np.max)

Unnamed: 0        Date  Confirmed
State    Country                                  
Zhejiang Germany      60446  2020-06-03     184121
         US           60554  2020-06-03    1851520

In [8]:
# this command will only work when adapting the get_doubling_time_via_regression function

#test_data.groupby(['state','country']).apply(get_doubling_time_via_regression)

In [25]:
def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result
    

In [26]:
test_data[['State','Country','Confirmed']].groupby(['State','Country']).apply(rolling_reg,'Confirmed')

State     Country       
Zhejiang  Germany  26704           NaN
                   27156           NaN
                   27605      7.417994
                   28058      7.142035
                   28509      8.012983
                   28961      6.954407
                   29413      6.501919
                   29866      7.390371
                   30318     10.136671
                   30772     13.541893
                   31222     13.781393
                   31675     13.140681
                   32126     12.037271
                   32579     12.735506
                   33032     16.052694
                   33481     21.372453
                   33934     27.427355
                   34388     27.511936
                   34840     21.792179
                   35289     21.495848
                   35742     26.565108
                   36193     36.198404
                   36645     43.982990
                   37097     49.423444
                   37551     74.043747


In [27]:
pd_DR_result=pd_JH_data[['State','Country','Confirmed']].groupby(['State','Country']).apply(rolling_reg,'Confirmed').reset_index()

In [28]:
pd_DR_result=pd_DR_result.rename(columns={'Confirmed':'confirmed_DR',
                             'level_2':'index'})
pd_DR_result.head()

,State,Country,index,confirmed_DR
0,Alberta,Canada,349,NaN
1,Alberta,Canada,801,NaN
2,Alberta,Canada,1253,NaN
3,Alberta,Canada,1705,NaN
4,Alberta,Canada,2158,NaN


In [29]:
pd_JH_data=pd_JH_data.reset_index()
pd_JH_data.head()

,index,Unnamed: 0,Date,State,Country,Confirmed
0,0,0,2020-01-22,NaN,Afghanistan,0
1,1,308,2020-01-22,Zhejiang,Cuba,0
2,2,307,2020-01-22,Zhejiang,Croatia,0
3,3,306,2020-01-22,Zhejiang,Cote d'Ivoire,0
4,4,305,2020-01-22,Zhejiang,Costa Rica,0


In [30]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','confirmed_DR']],on=['index'],how='left')
pd_result_larg.head()

,index,Unnamed: 0,Date,State,Country,Confirmed,confirmed_DR
0,0,0,2020-01-22,NaN,Afghanistan,0,NaN
1,1,308,2020-01-22,Zhejiang,Cuba,0,NaN
2,2,307,2020-01-22,Zhejiang,Croatia,0,NaN
3,3,306,2020-01-22,Zhejiang,Cote d'Ivoire,0,NaN
4,4,305,2020-01-22,Zhejiang,Costa Rica,0,NaN


In [31]:
#pd_result_larg[pd_result_larg['country']=='Germany']

# Filtering the data with groupby apply 

In [36]:
from scipy import signal

def savgol_filter(df_input,column='Confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result
        

In [37]:
pd_filtered_result=pd_JH_data[['State','Country','Confirmed']].groupby(['State','Country']).apply(savgol_filter).reset_index()

In [39]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','Confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

,index,Unnamed: 0,Date,State,Country,Confirmed,confirmed_DR,Confirmed_filtered
0,0,0,2020-01-22,NaN,Afghanistan,0,NaN,NaN
1,1,308,2020-01-22,Zhejiang,Cuba,0,NaN,0.0
2,2,307,2020-01-22,Zhejiang,Croatia,0,NaN,0.0
3,3,306,2020-01-22,Zhejiang,Cote d'Ivoire,0,NaN,0.0
4,4,305,2020-01-22,Zhejiang,Costa Rica,0,NaN,0.0


# Filtered doubling rate

In [40]:

pd_filtered_doubling=pd_result_larg[['State','Country','Confirmed_filtered']].groupby(['State','Country']).apply(rolling_reg,'Confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'Confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})

pd_filtered_doubling.tail()

,State,Country,index,confirmed_filtered_DR
35639,Zhejiang,Zimbabwe,58418,7.927461
35640,Zhejiang,Zimbabwe,58903,13.255208
35641,Zhejiang,Zimbabwe,59322,14.082051
35642,Zhejiang,Zimbabwe,59775,14.508642
35643,Zhejiang,Zimbabwe,60567,16.854839


In [41]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

,index,Unnamed: 0,Date,State,Country,Confirmed,confirmed_DR,Confirmed_filtered,confirmed_filtered_DR
60563,60563,60259,2020-06-03,NaN,San Marino,674,NaN,NaN,NaN
60564,60564,60258,2020-06-03,NaN,Saint Vincent and the Grenadines,26,NaN,NaN,NaN
60565,60565,60257,2020-06-03,NaN,Saint Lucia,18,NaN,NaN,NaN
60566,60566,60283,2020-06-03,NaN,Timor-Leste,24,NaN,NaN,NaN
60567,60567,60567,2020-06-03,Zhejiang,Zimbabwe,222,22.140351,221.4,16.854839


In [43]:
mask=pd_result_larg['Confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN) 


In [45]:
pd_result_larg[pd_result_larg['Country']=='Germany'].tail()

,index,Unnamed: 0,Date,State,Country,Confirmed,confirmed_DR,Confirmed_filtered,confirmed_filtered_DR
59455,59455,59276,2020-06-01,NaN,Germany,183594,NaN,NaN,NaN
59700,59700,59994,2020-06-02,Zhejiang,Germany,183879,783.060412,183871.9,776.311280
59907,59907,59728,2020-06-02,NaN,Germany,183879,NaN,NaN,NaN
60151,60151,60446,2020-06-03,Zhejiang,Germany,184121,697.778621,184105.2,788.135019
60358,60358,60180,2020-06-03,NaN,Germany,184121,NaN,NaN,NaN


In [46]:
pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)